In [53]:
import torch
from src.dataset_loaders import MS_Marco_RawDataLoader
from src.DataProcessors import Reranking_DataProcessor, Longformer_Reranking_DataProcessor
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [3]:
raw_data_loader = MS_Marco_RawDataLoader()

In [ ]:
raw_data_loader.collection[0]

In [61]:
from transformers import LongformerConfig, LongformerModel, LongformerForSequenceClassification
from transformers import LongformerTokenizer

In [62]:
tokenizer = LongformerTokenizer.from_pretrained("allenai/longformer-base-4096")

In [63]:
model = LongformerForSequenceClassification.from_pretrained('allenai/longformer-base-4096').to("cuda")

Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', '

In [64]:
tokenizer.vocab_size

50265

In [72]:
input_ids = tokenizer.encode('i my cat', return_tensors='pt').to("cuda")
input_ids

tensor([[   0,  118, 4157,  127, 4758,    2]], device='cuda:0')

In [73]:
outputs = model(input_ids)

In [74]:
outputs

(tensor([[ 0.0533, -0.1108]], device='cuda:0', grad_fn=<AddmmBackward>),)

In [1]:
from trec_car import read_data
import tqdm

In [ ]:
collection = {}
for page in tqdm.tqdm(read_data.iter_paragraphs(open('datasets/TREC_CAR/paragraphCorpus/dedup.articles-paragraphs.cbor', 'rb'))):
#     print(page.get_text())
#     print()
#     print(page.para_id)
    collection[page.para_id] = page.get_text()
#     break

25207848it [17:05, 23937.69it/s]

In [47]:
d_col = {1:"foo", 2:"bar", 3:"baz"}
q_col = {"q1":"qux", "q2":"ren"}
q_rels = {"q1":[1,3], "q2":[2]}

data_processor = Longformer_Reranking_DataProcessor(d_col, q_col, q_rels)

In [48]:
dataloader = data_processor.to_dataloader(2)

In [51]:
for batch in dataloader:
    print(batch)
    print(data_processor.decode_tensor(batch["input"]))

[{'input': [0, 2253, 1178, 646, 3388, 510, 742, 741, 1222, 2], 'label': [1]}, {'input': [0, 2558, 646, 3388, 510, 742, 48769, 2], 'label': [0]}]
{'input': tensor([[    0,     0],
        [ 2253,  2558],
        [ 1178,   646],
        [  646,  3388],
        [ 3388,   510],
        [  510,   742],
        [  742, 48769],
        [  741,     2],
        [ 1222,     1],
        [    2,     1]]), 'label': tensor([1, 0])}
['<s>qux [SEP] baz</s>', '<s>ren [SEP] foo</s><pad><pad>']


In [8]:
from pyserini.search import SimpleSearcher

searcher = SimpleSearcher('datasets/TREC_CAsT/CAsT_collection.index/')
searcher.set_bm25(0.9, 0.4)
searcher.set_rm3(10, 10, 0.5)

In [10]:
hits = searcher.search('hubble space telescope')

# Print the first 10 hits:
for i in range(0, 10):
    print(f'{i+1:2} {hits[i].docid:15} {hits[i].score:.5f}')

 1 MARCO_5161598   5.32090
 2 CAR_c9e8fce9db12bb3731161692c967eacaea9182d7 5.29080
 3 CAR_86967771c50dbc68811c4cf4ae2e5585dd73273d 5.25520
 4 MARCO_1245535   5.21250
 5 MARCO_2850821   5.20190
 6 MARCO_7459250   5.18180
 7 MARCO_5161594   5.12080
 8 MARCO_6293823   4.98080
 9 MARCO_5161592   4.97270
10 MARCO_5161596   4.96330


In [11]:
print(searcher.doc('MARCO_5161598').raw())

<HTML>
<BODY>
The Hubble Space Telescope (HST) is a space telescope that was launched into low Earth orbit in 1990, and remains in operation.With a 2.4-meter (7.9 ft) mirror, Hubble's four main instruments observe in the near ultraviolet, visible, and near infrared spectra.he Hubble Space Telescope (HST) is a space telescope that was launched into low Earth orbit in 1990, and remains in operation.
</BODY>
</HTML>
